In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from copy import copy
from tqdm import tqdm
import json
import pandas as pd

import torch
from transformers import BertModel, BertTokenizerFast
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from rouge import Rouge

### Hyperparameters

In [90]:
NUM_CLASSES = 4
MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 256

LR = 2e-5
TRAINING_BATCH_SIZE = 16
VAL_BATCH_SIZE = 1
EPOCHS = 4
DROPOUT = 0.3
NUM_WORKERS = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Read dataset

In [ ]:
loc = './dataset/'

train = pd.read_csv(loc + 'train.csv')
test = pd.read_json(loc + 'test.jsonl', lines=True)
val = pd.read_csv(loc + 'valid.csv')

print('Train dataset size: {}'.format(len(train)))
print('Test dataset size: {}'.format(len(test)))
print('Dev dataset size: {}'.format(len(val)))

**Define collate function for dataloader**

Need to define how to stack batches since different sentences can have different lengths

In [99]:
def collate_fcn(batch):
    ids = [x['id'] for x in batch]
    features = [x['features'] for x in batch]
    tokens_batch, input_ids_batch, input_masks_batch, token_type_ids_batch, p_len_batch, q_len_batch, a_len_batch = ([] for _ in range(7))
    for f_i in features:
        tokens, input_ids, input_masks, token_type_ids, p_len, q_len, a_len = ([] for _ in range(7))
        for f in f_i:
            tokens.append(f[0])
            input_ids.append(f[1])
            input_masks.append(f[2])
            token_type_ids.append(f[3])
            p_len.append(f[4])
            q_len.append(f[5])
            a_len.append(f[6])
        tokens_batch.append(tokens)
        input_ids_batch.append(input_ids)
        input_masks_batch.append(input_masks)
        token_type_ids_batch.append(token_type_ids)
        p_len_batch.append(p_len)
        q_len_batch.append(q_len)
        a_len_batch.append(a_len)
    input_ids_batch = torch.tensor(input_ids_batch, dtype=torch.long)
    input_masks_batch = torch.tensor(input_masks_batch, dtype=torch.long)
    token_type_ids_batch = torch.tensor(token_type_ids_batch, dtype=torch.long)
    p_len_batch = torch.tensor(p_len_batch, dtype=torch.long)
    q_len_batch = torch.tensor(q_len_batch, dtype=torch.long)
    a_len_batch = torch.tensor(a_len_batch, dtype=torch.long)
    labels = torch.tensor([x['label'] for x in batch], dtype=torch.long)

    return ids, tokens_batch, input_ids_batch, input_masks_batch, token_type_ids_batch, p_len_batch, q_len_batch, a_len_batch, labels

### Create Dataset

In [91]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, add_CLS=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.add_CLS = add_CLS

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        id = row['id']
        context = row['context']
        question = row['question']
        options = [row['answer0'], row['answer1'], row['answer2'], row['answer3']]
        correct_opt = row['label']
        
        feature_set = []
        context_tokens = self.tokenizer.tokenize(context)
        question_tokens = self.tokenizer.tokenize(question)
        for opt in options:
            opt_tokens = self.tokenizer.tokenize(opt)
            q_opt_tok = question_tokens + opt_tokens

            ''' [CLS] context [SEP] question option [SEP] '''
            ''' [ 0      0      0      1       1      1 ] '''
            tokens = ['[CLS]'] + context_tokens + ['[SEP]'] + q_opt_tok + ['[SEP]']
            token_type_ids = [0] * (len(context_tokens) + 2) + [1] * (len(q_opt_tok) + 1)
            
            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            input_masks = [1] * len(input_ids)

            # pad the tokens to max length
            padding_length = self.max_len - len(input_ids)
            padding = [0] * padding_length
            input_ids += padding
            input_masks += padding
            token_type_ids += padding

            assert len(input_ids) == self.max_len
            assert len(input_masks) == self.max_len
            assert len(token_type_ids) == self.max_len

            feature_set.append((tokens, input_ids, input_masks, token_type_ids,
                                len(context_tokens), len(question_tokens), len(opt_tokens)))

        return {'id': id,
                'features': feature_set,
                'label': correct_opt}

## Model architecture for Question-Answering
To enhance the context understanding ability of BERT fine-tuning, we perform multiway bidirectional attention over the BERT encoding output, refer to **COSMOS QA: Machine Reading Comprehension with Contextual Commonsense Reasoning** [(arxiv)](https://arxiv.org/pdf/1909.00277.pdf) 

In [ ]:
class BERT_multAttn(nn.Module):
    def __init__(self, dropout, hidden_size):
        super(BERT_multAttn, self).__init__()

        self.bert_encoder = BertModel.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(dropout)
        self.linear_tran = nn.Linear(hidden_size, hidden_size)
        self.linear_fuseP = nn.Linear(hidden_size * 2, hidden_size)
        self.linear_fuseQ = nn.Linear(hidden_size * 2, hidden_size)
        self.linear_fuseA = nn.Linear(hidden_size * 2, hidden_size)
        self.classifier = nn.Linear(hidden_size*3, NUM_CLASSES)

    def forward(self, input_ids, token_type_ids, attention_mask, p_len, q_len, a_len, labels):
        # flatten the inputs
        flat_input_ids = input_ids.view(-1, input_ids.shape[-1])
        flat_token_type_ids = token_type_ids.view(-1, token_type_ids.shape[-1])
        flat_attention_mask = attention_mask.view(-1, attention_mask.shape[-1])
        
        sequence_output, pooled_output = self.bert_encoder(input_ids=flat_input_ids,
                                                           token_type_ids=flat_token_type_ids,
                                                           attention_mask=flat_attention_mask,
                                                           output_all_encoded_layers=False)
        



**Initialize the pretrained tokenizer**

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

In [ ]:
train_dataset = Dataset(train, tokenizer, MAX_LEN)
val_dataset = Dataset(val, tokenizer, MAX_LEN)
test_dataset = Dataset(test, tokenizer, MAX_LEN)

In [ ]:
params_dataLoader = {'batch_size': TRAINING_BATCH_SIZE,
                     'shuffle': True,
                     'num_workers': NUM_WORKERS,
                     'collate_fn': collate_fcn}
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, **params_dataLoader)

In [104]:
for batch in train_dataset_loader:
    ids, tokens_batch, input_ids_batch, input_masks_batch, token_type_ids_batch, p_len_batch, q_len_batch, a_len_batch, labels = batch
    print(len(ids))
    print(len(tokens_batch))
    print(input_ids_batch.shape)
    print(input_masks_batch.shape)
    print(token_type_ids_batch.shape)
    print(p_len_batch.shape)
    print(q_len_batch.shape)
    print(a_len_batch.shape)
    print(labels.shape)
    dfsd

16
16
torch.Size([16, 4, 256])
torch.Size([16, 4, 256])
torch.Size([16, 4, 256])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16])


NameError: name 'dfsd' is not defined